In [1]:
# import dependencies 
import math
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter

In [2]:
# Graph Convolutional Network 
class GCN(torch.nn.Module):    
    def __init__(self, num_layers, feats_per_node, layer_1_feats, layer_2_feats, activation):
        super().__init__()
        self.activation = activation
        self.num_layers = num_layers

        self.w_list = nn.ParameterList()
        for i in range(self.num_layers):
            if i==0:
                w_i = Parameter(torch.Tensor(feats_per_node, layer_1_feats))
                self.reset_param(w_i)
            else:
                w_i = Parameter(torch.Tensor(layer_1_feats, layer_2_feats))
                self.reset_param(w_i)
            self.w_list.append(w_i)
            
    def forward(self, A_list, Nodes_list, nodes_mask_list):
        node_feats = Nodes_list[-1]
        #A_list: T, each element sparse tensor
        #take only last adj matrix in time
        Ahat = A_list[-1]
        #Ahat: NxN ~ 30k
        #sparse multiplication

        # Ahat NxN
        # self.node_embs = Nxk
        #
        # note(bwheatman, tfk): change order of matrix multiply
        last_l = self.activation(Ahat.matmul(node_feats.matmul(self.w_list[0])))
        for i in range(1, self.num_layers):
            last_l = self.activation(Ahat.matmul(last_l.matmul(self.w_list[i])))
        return last_l

    def reset_param(self, t):
        stdv = 2. / math.sqrt(t.size(0))
        t.data.uniform_(-stdv,stdv)

In [3]:
# WHAT WE NEED IN DATASET
# -> Edges 
# -> Node Feats
# -> Total Feats per node 

class Static_Node_Cls_Tasker:
    def __init__(self, dataset):
        self.data = dataset 
        self.num_classes = 2 
        self.adj_matrix = self.get_static_sp_adj(edges=self.data.edges)
        self.feats_per_node = dataset.feats_per_node 
        self.node_feats = self.data.node_feats 
        self.adj_matrix = self.normalize_adj(adj=self.adj_matrix, num_nodes=self.data.num_nodes)
        self.is_static = True        
    
    def get_static_sp_adj(self, edges):
        idx = edges["idx"]
        vals = torch.ones(idx.size(0),dtype=torch.long)
        return {"idx": idx, "vals": vals}

In [4]:
# Cross Entropy Class
class Cross_Entropy(torch.nn.Module):
    def __init__(self):
        super().__init__()
        #weights = torch.tensor(args.class_weights).to(args.device)

In [5]:
# Trainer Class 
class Trainer:
    def __init__(self, gcn):
        self.gcn = gcn         

In [6]:
tasker = Static_Node_Cls_Tasker(None)
# new instance of graph convolutional neural network 
# gcn = GCN(num_layers=2, 
#           feats_per_node=164, 
#           layer_1_feats=20, 
#           layer_2_feats=10, 
#           activation=torch.nn.RReLU()).to("cpu")
# trainer = Trainer(gcn=gcn)

AttributeError: 'NoneType' object has no attribute 'edges'